Import needed modules

In [1]:
import pandas as pd
import os
import subprocess

Import data

In [2]:
# read the data
TP_data = pd.read_csv("./data/TP_SIDERs_coor.csv", sep=",", header=0)
Software_data = pd.read_csv("./data/Results_Program_LmSIDERs_30-05-54.csv", sep=",", header=0)
print(f"'True Positives' row length: {TP_data.shape[0]}\n",
      f"Software row length: {Software_data.shape[0]}")

'True Positives' row length: 1606
 Software row length: 4886


In [3]:
# Declare bedops folders paths:
bedops_path_1 = "./bedops_data_LmSIDERs"
bedops_path_2 = "./bedops_data_LmSIDERs2"


Check how many elements for each chromosome in the "True Positive" data

In [4]:
TP_data["sseqid"].value_counts(sort=False)

sseqid
LinJ.01      7
LinJ.02     18
LinJ.03      8
LinJ.04     18
LinJ.05     18
LinJ.06     22
LinJ.07     27
LinJ.08     18
LinJ.09     21
LinJ.10     26
LinJ.11     39
LinJ.12     36
LinJ.13     22
LinJ.14     66
LinJ.15     25
LinJ.16     34
LinJ.17     34
LinJ.18     30
LinJ.19     40
LinJ.20     37
LinJ.21     39
LinJ.22     27
LinJ.23     60
LinJ.24     57
LinJ.25     42
LinJ.26     45
LinJ.27     63
LinJ.28     46
LinJ.29     74
LinJ.30     71
LinJ.31     80
LinJ.32     69
LinJ.33     85
LinJ.34     96
LinJ.35    110
LinJ.36     96
Name: count, dtype: int64

Check how many elements for each chromosome in the software data

In [5]:
Software_data["sseqid"].value_counts(sort=False)

sseqid
LinJ.01     31
LinJ.02     46
LinJ.03     22
LinJ.04     46
LinJ.05     54
LinJ.06     58
LinJ.07     67
LinJ.08    136
LinJ.09     75
LinJ.10     94
LinJ.11     79
LinJ.12    121
LinJ.13     77
LinJ.14     98
LinJ.15     61
LinJ.16     94
LinJ.17    101
LinJ.18    109
LinJ.19    143
LinJ.20     92
LinJ.21     89
LinJ.22    120
LinJ.23    150
LinJ.24    119
LinJ.25     94
LinJ.26    132
LinJ.27    172
LinJ.28    117
LinJ.29    276
LinJ.30    203
LinJ.31    279
LinJ.32    192
LinJ.33    242
LinJ.34    454
LinJ.35    309
LinJ.36    334
Name: count, dtype: int64

Difference in numbers in the software data compared to the "True Positives"

In [6]:
Software_data["sseqid"].value_counts(sort=False) - TP_data["sseqid"].value_counts(sort=False)

sseqid
LinJ.01     24
LinJ.02     28
LinJ.03     14
LinJ.04     28
LinJ.05     36
LinJ.06     36
LinJ.07     40
LinJ.08    118
LinJ.09     54
LinJ.10     68
LinJ.11     40
LinJ.12     85
LinJ.13     55
LinJ.14     32
LinJ.15     36
LinJ.16     60
LinJ.17     67
LinJ.18     79
LinJ.19    103
LinJ.20     55
LinJ.21     50
LinJ.22     93
LinJ.23     90
LinJ.24     62
LinJ.25     52
LinJ.26     87
LinJ.27    109
LinJ.28     71
LinJ.29    202
LinJ.30    132
LinJ.31    199
LinJ.32    123
LinJ.33    157
LinJ.34    358
LinJ.35    199
LinJ.36    238
Name: count, dtype: int64

Let's group data by chromosomes

In [7]:
TP_data_grouped = TP_data.groupby("sseqid")
Software_data_grouped = Software_data.groupby("sseqid")

Let's make the main code

In [8]:
# Check actual path
os.getcwd()

'/home/viskuit/Documents/Work_CBMSO/Leish_Data_Analysis/12.Sensibility_script'

In [9]:
def columns_to_numeric(data_input, columns_to_convert = []):
    for column in columns_to_convert:
        data_input[column] = pd.to_numeric(data_input[column], errors='coerce')
    return data_input

First option with different strands:

In [10]:
# Folder path
# bedops_path_1 = "./bedops_data"

comparison_dict = {}
for (name1, group1), (name2, group2) in zip(TP_data_grouped, Software_data_grouped):
    # group1 should be the TP data
    # ------------------------------------------------------------------------------
    path_chr = os.path.join(bedops_path_1, name1)
    os.makedirs(path_chr, exist_ok=True)
    # ------------------------------------------------------------------------------
    group1_plus, group1_minus = group1[group1["sstrand"] == "plus"].copy(), group1[group1["sstrand"] == "minus"].copy()
    group2_plus, group2_minus = group2[group2["sstrand"] == "plus"].copy(), group2[group2["sstrand"] == "minus"].copy()
    # ------------------------------------------------------------------------------
    group1_plus = group1_plus[["sseqid", "sstart", "send"]]
    group1_minus = group1_minus[["sseqid", "sstart", "send"]]
    group2_plus = group2_plus[["sseqid", "sstart", "send"]]
    group2_minus = group2_minus[["sseqid", "sstart", "send"]]
    # ------------------------------------------------------------------------------
    group1_minus[["sstart", "send"]] = group1_minus[["send", "sstart"]]  # because minus strand coordinates are reversed
    group2_minus[["sstart", "send"]] = group2_minus[["send", "sstart"]]  # because minus strand coordinates are reversed
    # ------------------------------------------------------------------------------
    path_group1_plus = os.path.join(path_chr, "group1_plus.bed")
    path_group1_minus = os.path.join(path_chr, "group1_minus.bed")
    path_group2_plus = os.path.join(path_chr, "group2_plus.bed")
    path_group2_minus = os.path.join(path_chr, "group2_minus.bed")
    # ------------------------------------------------------------------------------
    group1_plus.to_csv(path_group1_plus, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group1_minus.to_csv(path_group1_minus, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group2_plus.to_csv(path_group2_plus, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group2_minus.to_csv(path_group2_minus, sep="\t", header=False, index=False)  # tabular sep because of bed format
    # ------------------------------------------------------------------------------
    path_group1_list = [path_group1_plus, path_group1_minus]
    path_group2_list = [path_group2_plus, path_group2_minus]
    TP_df_list = [group1_plus, group1_minus]
    # ------------------------------------------------------------------------------
    for index, (path1, path2, TP_group) in enumerate(zip(path_group1_list, path_group2_list, TP_df_list)):  # There are the paths to the files
        condition = 1
        # print(f"group1:  {group1}\n"
        #       f"group2:  {group2}")
        cmd = f"bedops --element-of {condition} {path1} {path2}"
        result = subprocess.check_output(cmd, shell=True, universal_newlines=True)
        result_df = pd.DataFrame([x.split("\t") for x in result.split("\n") if x], 
                                 columns=["sseqid", "sstart", "send"])
        result_df = columns_to_numeric(result_df, ["sstart", "send"])  # defined function
        if index == 0:  # for the plus loop
            comparison_dict[name1] = [["plus", f"{result_df.shape[0]}/{TP_group.shape[0]}", f"{result_df.shape[0]/TP_group.shape[0]*100:.2f}:"]]
        else:  # for the minus loop
            comparison_dict[name1].append(["minus", f"{result_df.shape[0]}/{TP_group.shape[0]}", f"{result_df.shape[0]/TP_group.shape[0]*100:.2f}"])
comparison_dict

{'LinJ.01': [['plus', '2/3', '66.67:'], ['minus', '2/4', '50.00']],
 'LinJ.02': [['plus', '4/9', '44.44:'], ['minus', '3/9', '33.33']],
 'LinJ.03': [['plus', '5/6', '83.33:'], ['minus', '1/2', '50.00']],
 'LinJ.04': [['plus', '4/7', '57.14:'], ['minus', '9/11', '81.82']],
 'LinJ.05': [['plus', '6/13', '46.15:'], ['minus', '4/5', '80.00']],
 'LinJ.06': [['plus', '1/16', '6.25:'], ['minus', '3/6', '50.00']],
 'LinJ.07': [['plus', '9/19', '47.37:'], ['minus', '2/8', '25.00']],
 'LinJ.08': [['plus', '4/14', '28.57:'], ['minus', '4/4', '100.00']],
 'LinJ.09': [['plus', '3/11', '27.27:'], ['minus', '3/10', '30.00']],
 'LinJ.10': [['plus', '13/21', '61.90:'], ['minus', '4/5', '80.00']],
 'LinJ.11': [['plus', '7/33', '21.21:'], ['minus', '6/6', '100.00']],
 'LinJ.12': [['plus', '4/28', '14.29:'], ['minus', '1/8', '12.50']],
 'LinJ.13': [['plus', '3/3', '100.00:'], ['minus', '1/19', '5.26']],
 'LinJ.14': [['plus', '1/23', '4.35:'], ['minus', '1/43', '2.33']],
 'LinJ.15': [['plus', '6/12', '50.0

Second option without differenting strands

In [14]:
# Folder path, change it
# bedops_path = "./bedops_data2"
Total = 0
comparison_dict2 = {}
for (name1, group1), (name2, group2) in zip(TP_data_grouped, Software_data_grouped):
    # group1 should be the TP data
    # ------------------------------------------------------------------------------
    path_chr = os.path.join(bedops_path_2, name1)
    os.makedirs(path_chr, exist_ok=True)
    # ------------------------------------------------------------------------------
    group1_plus, group1_minus = group1[group1["sstrand"] == "plus"].copy(), group1[group1["sstrand"] == "minus"].copy()
    group2_plus, group2_minus = group2[group2["sstrand"] == "plus"].copy(), group2[group2["sstrand"] == "minus"].copy()
    # ------------------------------------------------------------------------------
    group1_plus = group1_plus[["sseqid", "sstart", "send"]]
    group1_minus = group1_minus[["sseqid", "sstart", "send"]]
    group2_plus = group2_plus[["sseqid", "sstart", "send"]]
    group2_minus = group2_minus[["sseqid", "sstart", "send"]]
    # ------------------------------------------------------------------------------
    group1_minus[["sstart", "send"]] = group1_minus[["send", "sstart"]]  # because minus strand coordinates are reversed
    group2_minus[["sstart", "send"]] = group2_minus[["send", "sstart"]]  # because minus strand coordinates are reversed
    # ------------------------------------------------------------------------------
    group1_total = pd.concat([group1_plus, group1_minus], axis=0); group1_total.sort_values(by=["sseqid", "sstart"], inplace=True)  # join the plus and minus strand data
    group2_total = pd.concat([group2_plus, group2_minus], axis=0); group2_total.sort_values(by=["sseqid", "sstart"], inplace=True)  # join the plus and minus strand data
    # ------------------------------------------------------------------------------
    path_group1_total = os.path.join(path_chr, "group1_total.bed")
    path_group2_total = os.path.join(path_chr, "group2_total.bed")
    # ------------------------------------------------------------------------------
    group1_total.to_csv(path_group1_total, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group2_total.to_csv(path_group2_total, sep="\t", header=False, index=False)  # tabular sep because of bed format

    # ------------------------------------------------------------------------------
    condition = 1
    cmd = f"bedops --element-of {condition} {path_group1_total} {path_group2_total}"
    result = subprocess.check_output(cmd, shell=True, universal_newlines=True)
    result_df = pd.DataFrame([x.split("\t") for x in result.split("\n") if x], 
                                columns=["sseqid", "sstart", "send"])
    result_df = columns_to_numeric(result_df, ["sstart", "send"])  # defined function
    comparison_dict2[name1] = [f"{result_df.shape[0]}/{group1_total.shape[0]}", f"{result_df.shape[0]/group1_total.shape[0]*100:.2f}"]
    Total += result_df.shape[0]
print(f"""
There are {Total} TP sequences of {TP_data.shape[0]} in LmSIDER data:
    - That's {Total/TP_data.shape[0]*100:.2f}% of the TP data
    - {TP_data.shape[0]-Total} TP sequences are not in LmSIDER data, which is {100-Total/TP_data.shape[0]*100:.2f}%
"""
)
comparison_dict2


There are 1596 TP sequences of 1606 in LmSIDER data:
    - That's 99.38% of the TP data
    - 10 TP sequences are not in LmSIDER data, which is 0.62%



{'LinJ.01': ['7/7', '100.00'],
 'LinJ.02': ['18/18', '100.00'],
 'LinJ.03': ['6/8', '75.00'],
 'LinJ.04': ['18/18', '100.00'],
 'LinJ.05': ['18/18', '100.00'],
 'LinJ.06': ['22/22', '100.00'],
 'LinJ.07': ['27/27', '100.00'],
 'LinJ.08': ['18/18', '100.00'],
 'LinJ.09': ['21/21', '100.00'],
 'LinJ.10': ['26/26', '100.00'],
 'LinJ.11': ['39/39', '100.00'],
 'LinJ.12': ['36/36', '100.00'],
 'LinJ.13': ['22/22', '100.00'],
 'LinJ.14': ['66/66', '100.00'],
 'LinJ.15': ['25/25', '100.00'],
 'LinJ.16': ['34/34', '100.00'],
 'LinJ.17': ['34/34', '100.00'],
 'LinJ.18': ['30/30', '100.00'],
 'LinJ.19': ['40/40', '100.00'],
 'LinJ.20': ['35/37', '94.59'],
 'LinJ.21': ['38/39', '97.44'],
 'LinJ.22': ['27/27', '100.00'],
 'LinJ.23': ['60/60', '100.00'],
 'LinJ.24': ['56/57', '98.25'],
 'LinJ.25': ['41/42', '97.62'],
 'LinJ.26': ['45/45', '100.00'],
 'LinJ.27': ['63/63', '100.00'],
 'LinJ.28': ['46/46', '100.00'],
 'LinJ.29': ['74/74', '100.00'],
 'LinJ.30': ['71/71', '100.00'],
 'LinJ.31': ['80/80